In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType,VarcharType,DateType,TimestampType
races_schema=spark.read.csv("/mnt/storageformula1training/raw/races.csv",header=True,inferSchema=True)

In [0]:
display(races_schema)

In [0]:
races_schema.printSchema()


In [0]:

races_schema_Struct=StructType(fields=[StructField("raceId",IntegerType(),False),
                                StructField("year",IntegerType(),True),
                                StructField("round",IntegerType(),True),
                                StructField("circuitId",IntegerType(),False),
                                StructField("name",StringType(),True),
                                StructField("date",DateType(),True),
                                StructField("time",StringType(),True),
                                StructField("url",StringType(),True)
                               ])

In [0]:

races_df=spark.read.csv("/mnt/storageformula1training/raw/races.csv",header=True,schema=races_schema_Struct)


In [0]:
races_df.printSchema()

In [0]:
from pyspark.sql.functions import col
races_df=races_df.select(col("raceId"),col("year"),col("round"),col("circuitId"),col("name"),col("date"),col("time"))

In [0]:
races_renamed_df=races_df.withColumnRenamed("raceId","race_id").withColumnRenamed("year","race_year").withColumnRenamed("round","race_round").withColumnRenamed("circuitId","circuit_id").withColumnRenamed("name","race_name").withColumnRenamed("date","race_date").withColumnRenamed("time","race_time")
display(races_renamed_df)   

In [0]:
from pyspark.sql.functions import to_timestamp,lit,concat
races_renamed_df=races_renamed_df.withColumn("race_timestamp",to_timestamp(concat(col("race_date"),lit(" "),col("race_time")),"yyyy-MM-dd HH:mm:ss"))

In [0]:
races_final_df=races_renamed_df.select(col("race_id"),col("race_year"),col("race_round"),col("circuit_id"),col("race_name"),col("race_timestamp"))
display(races_final_df)

In [0]:
from pyspark.sql.functions import current_timestamp
races_final_df=races_final_df.withColumn("ingestion_date",current_timestamp())

In [0]:
races_final_df.write.parquet("/mnt/storageformula1training/processed/races",mode="overwrite",partitionBy="race_year")

In [0]:
%fs
ls /mnt/storageformula1training/processed/races

In [0]:
races_processed=spark.read.parquet("/mnt/storageformula1training/processed/races")

In [0]:
display(races_processed)